In [ ]:
import cv2
import sys

def load_and_display_video(video_path):
    # Load the video
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        sys.exit()
    
    # Read and display frame by frame
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Reached the end of the video or an error occurred. Exiting.")
            break
        
        # Display the frame
        cv2.imshow('Video Frame', frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # When everything done, release the capture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Replace 'path_to_your_video.mp4' with the path to the video file you want to load
load_and_display_video('path_to_your_video.mp4')


In [ ]:
import cv2
import numpy as np

def preprocess_frame(frame, target_size=(224, 224), normalization=True):
    """
    Pre-process a single frame:
    - Resize frame
    - Normalize pixel values
    """
    # Resize the frame
    resized_frame = cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA)
    
    if normalization:
        # Normalize the frame's pixel values to be between 0 and 1
        normalized_frame = resized_frame / 255.0
        return normalized_frame
    else:
        return resized_frame

def select_key_frames(video_path, skip_frames=5):
    """
    Load video, and pre-process video frames, selecting key frames based on a skip value.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return []
    
    processed_frames = []
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Select key frames based on skip_frames
        if frame_count % skip_frames == 0:
            processed_frame = preprocess_frame(frame)
            processed_frames.append(processed_frame)
        
        frame_count += 1
    
    cap.release()
    return np.array(processed_frames)

# Example usage:
# Replace 'path_to_your_video.mp4' with the actual video file path
video_frames = select_key_frames('path_to_your_video.mp4', skip_frames=5)
print(f"Processed {len(video_frames)} frames.")


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

# Load MobileNetV2 with pre-trained ImageNet weights, excluding the top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define a model that will return the output of the base_model's last layer
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_features(video_frames):
    """
    Extract features for each frame in video_frames using MobileNetV2.
    """
    # Assuming video_frames is a 4D numpy array: (num_frames, height, width, channels)
    features = model.predict(video_frames)
    return features


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming we're working with a fixed feature size from MobileNetV2
feature_size = 7 * 7 * 1280  # This will depend on the output shape of your CNN
num_classes = 10  # Example number of classes for classification

# Define a simple LSTM model
lstm_model = Sequential([
    LSTM(256, input_shape=(None, feature_size)),  # Adjust based on the shape of the CNN output
    Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming `extracted_features` is a numpy array of shape (num_samples, num_frames, feature_size)
# and `labels` is a one-hot encoded numpy array of shape (num_samples, num_classes)
# extracted_features = extract_features(preprocessed_video_frames)  # From previous step
# labels = ...  # Load or define your labels here

# Fit the model
# lstm_model.fit(ex
